# Notebook 08 — Forecasting Monthly Medication-Error Counts

This notebook builds a simple Holt linear trend forecasting model.

The goal is to estimate the future direction of medication-error volume.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import Holt

file_path = "../data/Krista_240726_Final.xlsx"
med_df = pd.read_excel(file_path, sheet_name="Medication")
med_df.head()

## Prepare monthly time-series data

In [ ]:
# Convert Date column to datetime
med_df['Date'] = pd.to_datetime(med_df['Date'], errors='coerce')

# Drop rows without usable dates
med_df = med_df.dropna(subset=['Date'])

# Group by month
monthly_counts = med_df.groupby(med_df['Date'].dt.to_period('M')).size()
monthly_counts = monthly_counts.to_timestamp()

monthly_counts.head()

## Plot monthly counts

In [ ]:
monthly_counts.plot(figsize=(10,5), title="Monthly Medication-Error Counts")
plt.show()

## Fit Holt linear trend model

In [ ]:
holt_model = Holt(monthly_counts).fit()

# Forecast next 6 months
forecast = holt_model.forecast(6)
forecast

## Plot forecast results

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(monthly_counts, label='Historical')
plt.plot(forecast, label='Forecast', linestyle='--')
plt.title('Holt Forecast of Medication-Error Volume')
plt.legend()
plt.show()

## Compare actual vs fitted values

In [ ]:
fitted = holt_model.fittedvalues

plt.figure(figsize=(10,5))
plt.plot(monthly_counts, label='Actual')
plt.plot(fitted, label='Fitted', linestyle='--')
plt.title('Actual vs Fitted (Holt Model)')
plt.legend()
plt.show()